# Scube tutorial: 3D expression modeling with gaussian process regression
# 使用高斯过程回归进行 3D 表达建模

Dataset: 3D STARmap of mouse brain ([here](https://zenodo.org/record/8167488)).

注意：如果要在 Scube 中运行 3D 表达 GPR 模型，需要先安装 [Open3D](http://www.open3d.org/docs/release/) python 库。

In [13]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Scube
from SPACEL.Scube.utils_3d import create_mesh, smooth_mesh, sample_in_mesh, get_surface_colors, save_view_parameters, load_view_parameters
import scanpy as sc
import numpy as np
import pandas as pd
np.random.seed(42)

Setting environment seed: 42


## Load data

3D 表达模型的输入数据类型是归一化表达和每个 spot/cell 的 3D 坐标。

In [14]:
st_ad = sc.read_h5ad('data/starmap_3d_mouse_brain/starmap_3d_mouse_brain.h5ad')
sc.pp.normalize_total(st_ad,target_sum=1e4)
sc.pp.log1p(st_ad)

# Normalized expression data (rows as spots/cells, columns as genes)
norm_expr = st_ad.to_df()

# 3D location (rows as spots/cells, columns as x,y,z axis coordinate)
loc = st_ad.obsm['spatial'].copy()

## Generate 3D surface

在此步骤中，我们使用 alpha 形状方法从 spots/cells 的 3D 位置坐标创建三角形网格。默认情况下，参数 α 是自动估计的，它会影响网格中三角形的数量。

In [19]:
mesh = create_mesh(loc.values.reshape(-1,3), alpha=80, show=False)

alpha=80.000


为了获得平滑的网格，我们可以对网格使用 Taubin 滤波器和网格细分。通过增加 `taubin_iter` 和 `subdivide_iter` 将生成更平滑的网格。

In [20]:
mesh = smooth_mesh(mesh,taubin_iter=10,subdivide_iter=3,show=False)

filter with Taubin with 10 iterations
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [Subdivid

从网格内部采样 500000 个位置坐标并获取网格表面的顶点。

In [22]:
loc_sampled,loc_surface = sample_in_mesh(mesh,loc.values.reshape(-1,3))

## Gaussian process regression

为了使 GPR 模型具有更好的性能，我们建议将位置坐标值调整为 0 到 10 左右。

In [23]:
loc = loc/1000
loc_sampled = loc_sampled/1000

然后，我们构建 GPR 模型并对其进行训练直至收敛。如果 GPU 设备可用，请将 `use_gpu` 设置为 `True`

In [24]:
gpr_model = Scube.GPRmodel(norm_expr,loc.values,loc_sampled,used_genes=norm_expr.columns,use_gpu=False)

In [25]:
gpr_model.train(lr=.02,training_iter=50,save_model=True,cal_bf=True,optim_l=True,optimize_method='Adam')

Modeling Slc17a7
Optimize lenthscale...


/home/zhenghu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/SPACEL/Scube/gpr.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  constant = torch.tensor(self.train_y.mean())


Initialize lenthscale alpha as 1.000
1.6126749515533447
1.6040093898773193
1.595556378364563
1.5873961448669434
1.580487847328186
1.5730875730514526
1.5651644468307495
1.557767629623413
1.5520429611206055
1.5451592206954956
1.5391695499420166
1.534324049949646
1.528257966041565
1.5231934785842896
1.517529845237732
1.5143438577651978
1.5099422931671143
1.504738688468933
1.4995595216751099
1.496253490447998
1.4913825988769531
1.4881261587142944
1.4835671186447144
1.479087471961975
1.4755985736846924
1.4716520309448242
1.4681977033615112
1.4646929502487183
1.4585686922073364
1.4552431106567383
1.451104760169983
1.4450016021728516
1.442122220993042
1.436931848526001
1.433657169342041
1.4269747734069824
1.423779010772705
1.420331597328186
1.4163175821304321
1.415096640586853
1.4138773679733276
1.409180998802185
1.4060618877410889
1.4017099142074585
1.3998111486434937
1.391788125038147
1.3922747373580933
1.3870528936386108
1.379839539527893
1.3747437000274658
Best model loss: 1.3747437000274

/home/zhenghu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(


1.7354427576065063
1.7266426086425781
1.718192219734192
1.7100541591644287
nan
1.6945265531539917
1.6871986389160156


/home/zhenghu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/linear_operator/operators/added_diag_linear_operator.py:128: NumericalWarning: NaNs encountered in preconditioner computation. Attempting to continue without preconditioning.
  warnings.warn(


1.6792595386505127
nan
1.6668370962142944
1.6605945825576782
1.6546218395233154


/home/zhenghu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(


1.64892578125
1.6433374881744385
1.63814377784729
1.6337909698486328
1.6278345584869385
1.6236587762832642
1.6193121671676636
1.6150442361831665
1.6108077764511108
1.6073542833328247
1.6036937236785889
1.6002157926559448
1.5969430208206177
1.5938400030136108
1.5908828973770142
1.5882463455200195
1.5853468179702759
1.5827966928482056
1.580336570739746
1.5780810117721558
1.5758651494979858
nan
1.5721203088760376
1.569946050643921
1.5679659843444824
1.5664937496185303
1.5650888681411743
1.5634033679962158
1.5619534254074097
1.5606117248535156
1.5593129396438599
1.5581296682357788
1.5569626092910767
1.555999755859375
1.554781198501587
1.5538082122802734
Best model loss: 1.5538082122802734
0.17906451225280762
Modeling Mgp
Optimize lenthscale...
Initialize lenthscale alpha as 1.000
0.8693720698356628
0.8743863105773926
0.8685742020606995
0.8560292720794678
0.8524212837219238
0.8487515449523926
0.8408825397491455
0.8392318487167358
0.8336750268936157
0.8470943570137024
0.8304954171180725
0.83

通过计算贝叶斯因子，我们可以统计识别 3D 表达模型中空间变化的基因。对于基因来说，贝叶斯因子值越大表示空间变异越大。

## Visualization

任何基因的 GRP 模型结果的可视化。

In [26]:
g = 'Cux2'

# Expression of sampled location from the 3D mesh
sampled_expr = gpr_model.plot_gpr_expr(g,s=.1,training_iter=50,lr=.02,save=False,show=True,elev=45,azim=45,cmap='Spectral_r',zlim=(-0.3,0.4))

# Expression of surface location from the 3D mesh
surface_expr = gpr_model.plot_gpr_expr(g,data=loc_surface/1000,s=.1,training_iter=50,lr=.02,save=False,show=True,elev=45,azim=45,cmap='Spectral_r',zlim=(-0.3,0.4))

# Expression of measurement location from the observation
Scube.plot_3d(loc=loc.values,val=norm_expr[g],s=.1,show=True,elev=45,azim=45,zlim=(-0.3,0.4))

/home/zhenghu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/SPACEL/Scube/gpr.py:302: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  observed_pred_batch = self.model.likelihood(self.model(torch.tensor(data_batch)))


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 2 but got size 3 for tensor number 1 in the list.

此步骤用 GPR 表达对网格表面进行着色。它可用于通过 `Open3D` 库对 GRP 模型结果进行可视化。

In [ ]:
get_surface_colors(mesh,surface_expr)

将相机视图参数保存在 json 文件中，然后在下一次可视化时加载它。

In [ ]:
save_view_parameters(mesh, 'view_parameters.json')

In [ ]:
g = 'Mbp'
# Expression of surface location from the 3D mesh
surface_expr = gpr_model.plot_gpr_expr(g,data=loc_surface/1000,s=.1,training_iter=50,lr=.02,save=False,show=False,elev=45,azim=45,cmap='Spectral_r',zlim=(-0.3,0.4))
get_surface_colors(mesh,surface_expr)
load_view_parameters(mesh, 'view_parameters.json')